In [2]:
import pandas as pd
import numpy as np 
from scipy.stats import skew
import klib

In [3]:
df = pd.read_csv(r"D:\2025 lessons\AI+ML course amaliyot\Datasets\housing_in_london.csv")
df.head()                       # * target varialbe is average_price

,date,area,average_price,code,houses_sold,no_of_crimes,borough_flag
0,1995-01-01,city of london,91449,E09000001,17.0,NaN,1
1,1995-02-01,city of london,82203,E09000001,7.0,NaN,1
2,1995-03-01,city of london,79121,E09000001,14.0,NaN,1
3,1995-04-01,city of london,77101,E09000001,7.0,NaN,1
4,1995-05-01,city of london,84409,E09000001,10.0,NaN,1


In [4]:
df = klib.data_cleaning(df) 
df.info()

Shape of cleaned data: (13549, 7) - Remaining NAs: 6204


Dropped rows: 0
     of which 0 duplicates. (Rows (first 150 shown): [])

Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 0.49 MB (-68.06%)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13549 entries, 0 to 13548
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   date           13549 non-null  category
 1   area           13549 non-null  category
 2   average_price  13549 non-null  int32   
 3   code           13549 non-null  category
 4   houses_sold    13455 non-null  float32 
 5   no_of_crimes   7439 non-null   float32 
 6   borough_flag   13549 non-null  int8    
dtypes: category(3), float32(2), int32(1), int8(1)
memory usage: 238.3 KB


In [5]:
for col in df.columns[df.isnull().any()]:     # ?  missing value  numeric type features ichida bulganligi sababli 
    df[col] = df[col].fillna(df[col].mean())    # ! fill missing values

df.isnull().sum()

date             0
area             0
average_price    0
code             0
houses_sold      0
no_of_crimes     0
borough_flag     0
dtype: int64

# Encoding

In [6]:
from sklearn.preprocessing import LabelEncoder

df.drop(columns = ['code'], inplace=True)
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13549 entries, 0 to 13548
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   date           13549 non-null  category
 1   area           13549 non-null  category
 2   average_price  13549 non-null  int32   
 3   houses_sold    13549 non-null  float32 
 4   no_of_crimes   13549 non-null  float32 
 5   borough_flag   13549 non-null  int8    
dtypes: category(2), float32(2), int32(1), int8(1)
memory usage: 223.7 KB


In [7]:
category_col= df.select_dtypes(include='category').columns
df_encoded = df.copy(deep=True)
for col in category_col:
    df_encoded[col] = LabelEncoder().fit_transform(df[col])
df_encoded.head()

,date,area,average_price,houses_sold,no_of_crimes,borough_flag
0,0,6,91449,17.0,2158.352051,1
1,1,6,82203,7.0,2158.352051,1
2,2,6,79121,14.0,2158.352051,1
3,3,6,77101,7.0,2158.352051,1
4,4,6,84409,10.0,2158.352051,1


# Without scaling

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error

model = RandomForestRegressor(n_estimators=100,random_state=42)

In [9]:
# df_noscale = df.copy(deep=True)

y_ns = df_encoded['average_price'] #   No scaling
x_ns = df_encoded.drop(columns=['average_price'])





In [10]:
x_train_ns ,x_temp_ns,y_train_ns,y_temp_ns=train_test_split(x_ns,y_ns,test_size=0.2,random_state=42)
x_test_ns,x_val_ns,y_test_ns,y_val_ns=train_test_split(x_temp_ns,y_temp_ns,test_size=0.5,random_state=42)

noscale_model = model.fit(x_train_ns,y_train_ns)
noscale_model


RandomForestRegressor(random_state=42)

In [11]:
y_pred_ns=noscale_model.predict(x_test_ns)
mse_noscale=mean_squared_error(y_test_ns,y_pred_ns)
r2_noscale=r2_score(y_test_ns,y_pred_ns)
mse_noscale




422029712.3357187

# With Scaling

In [12]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
df.head()

,date,area,average_price,houses_sold,no_of_crimes,borough_flag
0,1995-01-01,city of london,91449,17.0,2158.352051,1
1,1995-02-01,city of london,82203,7.0,2158.352051,1
2,1995-03-01,city of london,79121,14.0,2158.352051,1
3,1995-04-01,city of london,77101,7.0,2158.352051,1
4,1995-05-01,city of london,84409,10.0,2158.352051,1


In [13]:
df_scaled = df.copy(deep=True)
df_scaled['date']=df_scaled['date'].str.split('(').str[0].str.strip()
df_scaled['date'] = pd.to_datetime(df_scaled['date'],errors= 'coerce')
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_scaled)
df_scaled = pd.DataFrame(MinMaxScaler().fit_transform(df),columns=df_scaled.columns)

df.head()

TypeError: float() argument must be a string or a real number, not 'Timestamp'

In [ ]:
y_scaled = df_scaled['average_price']
x_scaled = df_scaled.drop(columns='average_price')


In [ ]:
x_train_sc ,x_temp_sc,y_train_sc,y_temp_sc=train_test_split(x_scaled,y_scaled,test_size=0.2,random_state=42)
x_test_sc,x_val_sc,y_test_sc,y_val_sc=train_test_split(x_temp_sc,y_temp_sc,test_size=0.5,random_state=42)

scaled_model = model.fit(x_train_sc,y_train_sc)
scaled_model


RandomForestRegressor(random_state=42)

In [ ]:
y_pred_sc=noscale_model.predict(x_test_sc)
mse_scaled=mean_squared_error(y_test_sc,y_pred_sc)
r2_scaled=r2_score(y_test_sc,y_pred_sc)

In [ ]:
mse_scaled

0.012008736501061892

# Feature Engineering

In [ ]:
df.head()

,date,area,average_price,houses_sold,no_of_crimes,borough_flag
0,0,6,91449,17.0,2158.352051,1
1,1,6,82203,7.0,2158.352051,1
2,2,6,79121,14.0,2158.352051,1
3,3,6,77101,7.0,2158.352051,1
4,4,6,84409,10.0,2158.352051,1
